In [68]:
# Reference used: https://www.geeksforgeeks.org/tf-idf-for-bigrams-trigrams/
import nltk 
import re 
import math
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import normalize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import pandas as pd   
import jsonlines
import json
from tqdm.notebook import tqdm
import pickle

In [23]:
# nltk.download('punkt')
# nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/raghav/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
documents = []
docID = {}
term_total_citations = {}
author_term_citations = {}
idx = 0
with jsonlines.open('data_india_sample.jl') as reader:
    for obj in tqdm(reader, leave=False):
        documents.append(obj)
        docID[idx] = obj['user']
        author_term_citations[obj['user']] = {}
        idx += 1

0it [00:00, ?it/s]

In [16]:
print(documents)

[{'country': 'india', 'org': '10153116334818526581', 'institute': 'National Institute of Technology Agartala', 'name': 'A K Chakraborty', 'user': 'cZxrxCMAAAAJ', 'homepage': 'Not Found', 'scholarPage': 'https://scholar.google.co.in/citations?user=cZxrxCMAAAAJ&hl=en', 'imgLink': 'https://scholar.google.co.in/citations/images/avatar_scholar_128.png', 'verifiedEmail': True, 'subjects': ['Power System', 'Smart Grid'], 'yearCitations': ['{"year": 2009, "citations": 3}', '{"year": 2010, "citations": 9}', '{"year": 2011, "citations": 14}', '{"year": 2012, "citations": 17}', '{"year": 2013, "citations": 32}', '{"year": 2014, "citations": 26}', '{"year": 2015, "citations": 49}', '{"year": 2016, "citations": 67}', '{"year": 2017, "citations": 62}', '{"year": 2018, "citations": 92}', '{"year": 2019, "citations": 94}', '{"year": 2020, "citations": 129}', '{"year": 2021, "citations": 42}'], 'citationsAll': '645', 'citations2016': '487', 'h-indexAll': '13', 'h-index2016': '10', 'i10-indexAll': '17',

In [29]:
# Code Taken From: https://www.geeksforgeeks.org/tf-idf-for-bigrams-trigrams/
def remove_string_special_characters(s):
    # removes special characters with ' '
    stripped = re.sub('[^a-zA-z\s]', ' ', s)
#     print(stripped)
    stripped = re.sub('_', ' ', stripped)
      
    # Change any white space to one space
    stripped = re.sub('\s+', ' ', stripped)
      
    # Remove start and end white spaces
    stripped = stripped.strip()
    if stripped != '':
            return stripped.lower()

In [12]:
# str="hello_world$2 there fnrAie"
# re.findall("[^a-z\s]",str)
print(remove_string_special_characters("hello_wordl"))

hello wordl
hello wordl


In [18]:
# Code Reference: https://www.geeksforgeeks.org/python-stemming-words-with-nltk/
def stem_string(sentence):
    ps = PorterStemmer()
    
    words = word_tokenize(sentence)
    words = [ps.stem(word) for word in words]
    return " ".join(words)

In [19]:
def remove_stop_words(sentence):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(sentence)
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

In [20]:
def process_string(sentence):
    sentence = remove_string_special_characters(sentence)
    sentence = stem_string(sentence)
    sentence = remove_stop_words(sentence)
    return sentence

### TF-IDF Matrix

In [21]:
def update_term_citations(document, sentence, numCitations, userID):
    numCitations += 1
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-zA-Z0-9\s]', ' ', sentence)
    tokens = [token for token in sentence.split(" ") if token != ""]
    for n in range(1, 5):
#         seen_ngms = []
        ngms = list(ngrams(tokens, n))
        for ngm in ngms:
            gm = " ".join(ngm)
#             if gm in seen_ngms:
#                 continue
#             seen_ngms.append(gm)
            if gm in author_term_citations[userID]:
                author_term_citations[userID][gm] += numCitations
            else:
                author_term_citations[userID][gm] = numCitations
            if gm in term_total_citations:
                term_total_citations[gm] += numCitations
            else:
                term_total_citations[gm] = numCitations

{"country": "india", 
 "org": "10153116334818526581", "institute": "National Institute of Technology Agartala", "name": "A K Chakraborty", "user": "cZxrxCMAAAAJ", "homepage": "Not Found", "scholarPage": "https://scholar.google.co.in/citations?user=cZxrxCMAAAAJ&hl=en", "imgLink": "https://scholar.google.co.in/citations/images/avatar_scholar_128.png", "verifiedEmail": true, "subjects": ["Power System", "Smart Grid"], "yearCitations": ["{\"year\": 2009, \"citations\": 3}", "{\"year\": 2010, \"citations\": 9}", "{\"year\": 2011, \"citations\": 14}", "{\"year\": 2012, \"citations\": 17}", "{\"year\": 2013, \"citations\": 32}", "{\"year\": 2014, \"citations\": 26}", "{\"year\": 2015, \"citations\": 49}", "{\"year\": 2016, \"citations\": 67}", "{\"year\": 2017, \"citations\": 62}", "{\"year\": 2018, \"citations\": 92}", "{\"year\": 2019, \"citations\": 94}", "{\"year\": 2020, \"citations\": 129}", "{\"year\": 2021, \"citations\": 42}"], "citationsAll": "645", "citations2016": "487", "h-indexAll": "13", "h-index2016": "10", "i10-indexAll": "17", "i10-index2016": "11", "papers": [{"title": "Effects of ambient temperature and wind speed on performance of monocrystalline solar photovoltaic module in Tripura, India", "authors": "T Bhattacharya, AK Chakraborty, K Pal", "conference": "Journal of Solar Energy 2014", "citations": 68, "year": 2014}, {"title": "Solution of optimal power flow using non dominated sorting multi objective opposition based gravitational search algorithm", "authors": "AR Bhowmik, AK Chakraborty", "conference": "International Journal of Electrical Power & Energy Systems 64, 1237-1250", "citations": 62, "year": 2015}, {"title": "Polarization encoded all-optical quaternary R\u2013S flip-flop using binary latch", "authors": "T Chattopadhyay, JN Roy, AK Chakraborty", "conference": "Optics Communications 282 (7), 1287-1293", "citations": 56, "year": 2009}, {"title": "Solution of optimal power flow using nondominated sorting multi objective gravitational search algorithm", "authors": "AR Bhowmik, AK Chakraborty", "conference": "International Journal of Electrical Power & Energy Systems 62, 323-334", "citations": 49, "year": 2014}, {"title": "Solution of short-term hydrothermal scheduling using sine cosine algorithm", "authors": "S Das, A Bhattacharya, AK Chakraborty", "conference": "Soft Computing 22 (19), 6409-6427", "citations": 37, "year": 2018}, {"title": "Performance analysis and comparison of various FACTS devices in power system", "authors": "A Saha, P Das, AK Chakraborty", "conference": "International Journal of Computer Applications 46 (15), 0975-0887", "citations": 36, "year": 2012}, {"title": "Active power loss minimization with FACTS devices using SA/PSO techniques.", "authors": "S Majumdar, AK Chakraborty, PK Chattopadhyay", "conference": "2009 International Conference on Power Systems, 1-5", "citations": 21, "year": 2009}, {"title": "Optimal location of UPFC based on PSO algorithm considering active power loss minimization", "authors": "AR Bhowmik, AK Chakraborty, P Das", "conference": "2012 IEEE Fifth Power India Conference, 1-5", "citations": 17, "year": 2012}, {"title": "Fixed head short-term hydrothermal scheduling in presence of solar and wind power", "authors": "S Das, A Bhattacharya, AK Chakraborty", "conference": "Energy strategy reviews 22, 47-60", "citations": 15, "year": 2018}, {"title": "Water evaporation algorithm: a new metaheuristic algorithm towards the solution of optimal power flow", "authors": "A Saha, P Das, AK Chakraborty", "conference": "Engineering Science and Technology, an International Journal 20 (6), 1540-1552", "citations": 15, "year": 2017}, {"title": "Quasi-reflected ions motion optimization algorithm for short-term hydrothermal scheduling", "authors": "S Das, A Bhattacharya, AK Chakraborty", "conference": "Neural Computing and Applications 29 (6), 123-149", "citations": 14, "year": 2018}, {"title": "Advanced metering infrastructure: Technology and challenges", "authors": "AK Chakraborty, N Sharma", "conference": "2016 IEEE/PES Transmission and Distribution Conference and Exposition (T&D), 1-5", "citations": 13, "year": 2016}, {"title": "Distribution and determinants of cytomegalovirus induced end organ disease/s among people living with HIV/AIDS in a poor resource setting: observation from India", "authors": "A Chakraborty, T Mahapatra, S Mahapatra, S Ansari, S Siddhanta, ...", "conference": "PloS one 10 (2), e0117466", "citations": 13, "year": 2015}, {"title": "Network analysis and synthesis", "authors": "SP Ghosh, AK Chakraborty", "conference": "Tata McGraw Hill", "citations": 13, "year": 2010}, {"title": "Neutral Current and Neutral Voltage in Three Phase Four Wire Distribution System of a Technical Institution", "authors": "N Dey, AK Chakraborty", "conference": "International Journal of Computer Applications 72 (3)", "citations": 10, "year": 2013}, {"title": "Congestion management in a deregulated power system by rescheduling of sensitive generators and load curtailment using PSO", "authors": "T Bhattacharjee, AK Chakraborty", "conference": "International Journal of Emerging Technology and Advanced Engineering 2 (3\u00a0\u2026", "citations": 10, "year": 2012}, {"title": "Zinc and vitamin A can minimise the severity of oral submucous fibrosis", "authors": "R Dhariwal, S Mukherjee, S Pattanayak, A Chakraborty, JG Ray, ...", "conference": "Case Reports 2010, bcr1020092348", "citations": 10, "year": 2010}, {"title": "Control of permanent magnet synchronous motor (pmsm) using vector control approach", "authors": "AK Chakraborty, N Sharma", "conference": "2016 IEEE/PES Transmission and Distribution Conference and Exposition (T&D), 1-5", "citations": 9, "year": 2016}, {"title": "Active line flow control of power system network with FACTS devices of choice using soft computing technique", "authors": "AK Chakraborty, S Majumdar, PK Chattopadhyay, C Nandi", "conference": "International Journal of Computer Applications 975, 8887", "citations": 9, "year": 2011}, {"title": "Computer simulation of the influence of shading on a solar photovoltaic array", "authors": "T Bhattacharya, AK Chakraborty, K Pal", "conference": "International Journal of Ambient Energy 38 (5), 525-533", "citations": 8, "year": 2017}]}

In [31]:
word_documents = []
for document in tqdm(documents, leave=False):
    sentences = []
    for subject in document['subjects']:
        sentence = process_string(subject)
        sentences.append(sentence)
        update_term_citations(document, sentence, int(document['citationsAll']), document['user'])
    for paper in document['papers']:
        try:
            sentence = paper['title'] + " " + paper['conference']
            sentence = process_string(sentence)
        except: 
            continue
#         print(sentence)
#         print(sentence)
        update_term_citations(document, sentence, paper['citations'], document['user'])
        sentences.append(sentence)
        
#     print("****************")
#     print(sentences)
#     print("****************")
#     word_documents.append(sentences)
    word_documents.append(".\n".join(sentences))
#     print(word_documents[-1])
#     word_documents.append(sentences)
#     print(".".join(sentences))

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# class tfidf:
#     def __init__():
        

In [78]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,4),min_df=0, analyzer='word')
tfidf_matrix = vectorizer.fit_transform(word_documents) 
features = (vectorizer.get_feature_names())
idf_scores = vectorizer.idf_

cnt_vct = CountVectorizer(stop_words='english', ngram_range=(1,4),min_df=0, analyzer='word')
cnt_matrix = cnt_vct.fit_transform(word_documents)
cnt_features = cnt_vct.get_feature_names()

cnt_matrix_arr = cnt_matrix.toarray()
tfidf_new_matrix = cnt_matrix_arr*idf_scores

In [92]:
query="data mining"
# print(features[3730])
query = process_string(query)
cnt_matrix_test = cnt_vct.transform([query])
cnt_matrix_test_arr = cnt_matrix_test.toarray()

# print(tfidf_new_matrix.shape)
# print(cnt_matrix_test_arr.shape)

res = np.matmul(tfidf_new_matrix,cnt_matrix_test_arr.T).squeeze()
# print(res)
# for i in range(res.size):
#     print(docID[i])
related_docs_indices = res.argsort()[:-10:-1]
related_results = [docID[i] for i in related_docs_indices]
print(related_results)


# cosine_similarities = linear_kernel(result_matrix, loaded_matrix).flatten()
# related_docs_indices = cosine_similarities.argsort()[:-10:-1]
# related_results = [docID[i] for i in related_docs_indices]
# print(related_results)

['laT3mwoAAAAJ', 'Pp5WeBsAAAAJ', 'BJ7CzPgAAAAJ', 'CmBE6P0AAAAJ', 'eeHeiSAAAAAJ', 'u_nJKdwAAAAJ', 'BaoAeqQAAAAJ', '1igRo6wAAAAJ', 'owqaSZQAAAAJ']


In [10]:
# print(tfidf_matrix)
# print(tfidf_matrix[(0, 3235)])

In [11]:
# print(author_term_citations)
# feature_idx = {}
# for idx, feature in enumerate(features):
#     feature_idx[feature] = idx
# author_idx = 0
# for author, author_dict in tqdm(author_term_citations.items(), leave=False):
#     for term, term_citations in author_dict.items():
#         if term in feature_idx:
#             term_idx = feature_idx[term]
#             log_factor = 1 + math.log( 1 + ((term_citations) / (term_total_citations[term] + 1)) , 2)
# #             print(log_factor)
#             tfidf_matrix[(author_idx, term_idx)] *= log_factor
#     author_idx += 1

In [12]:
# tfidf_matrix = normalize(tfidf_matrix, norm='l2', axis=0)

### Pickle the Matrix and Vectorizer

In [13]:
pickle.dump(vectorizer, open("web_data/tfidf_vectorizer.pkl", "wb"))
pickle.dump(tfidf_matrix, open("web_data/tfidf_matrix.pkl", "wb"))
pickle.dump(docID, open("web_data/doc_id_dict.pkl", "wb"))

### Query Subjects

In [44]:
query="data mining"
# print(features[3730])
query = process_string(query)
result_matrix = vectorizer.transform([query])
print(result_matrix.shape)
print(result_matrix)
cosine_similarities = linear_kernel(result_matrix, tfidf_matrix).flatten()
print(len(cosine_similarities))
related_docs_indices = cosine_similarities.argsort()[:-10:-1]
print(related_docs_indices)
related_results = [docID[i] for i in related_docs_indices]
print(related_results)
# print(result_matrix)

(1, 5261)
  (0, 1083)	1.0
10
[9 8 7 6 5 4 3 2 1]
['laT3mwoAAAAJ', 'Pp5WeBsAAAAJ', 'BJ7CzPgAAAAJ', 'CmBE6P0AAAAJ', 'eeHeiSAAAAAJ', 'u_nJKdwAAAAJ', 'BaoAeqQAAAAJ', '1igRo6wAAAAJ', 'owqaSZQAAAAJ']


### Activeness over 'n' years

In [15]:
for doc in doccuments:
    
    
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-15-36ec8c92a847>, line 5)

### Pickled Testing

In [ ]:
loaded_vectorizer = pickle.load(open("web_data/tfidf_vectorizer.pkl","rb"))
loaded_matrix = pickle.load(open("web_data/tfidf_matrix.pkl","rb"))
query="data mining"
# print(features[3730])
query = process_string(query)
result_matrix = loaded_vectorizer.transform([query])
cosine_similarities = linear_kernel(result_matrix, loaded_matrix).flatten()
related_docs_indices = cosine_similarities.argsort()[:-10:-1]
related_results = [docID[i] for i in related_docs_indices]
print(related_results)